In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option("display.max_columns",None) 
pd.set_option("display.max_rows",None) 
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
sns.set(style="darkgrid", palette="pastel", color_codes=True)
sns.set_context("paper")
from datetime import datetime
#import datapane as dp

#Bokeh imports
from bokeh.io import output_file, show, output_notebook, save
from bokeh.models import ColumnDataSource, Select, DateRangeSlider
from bokeh.plotting import figure, show
from bokeh.models import CustomJS 
from bokeh.layouts import row,column
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv('./archive/covid_19_clean_complete.csv')

In [3]:
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [4]:
#Renaming the coulmns for easy usage
df.rename(columns={'Date': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Lat':'lat', 'Long':'long',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)

# Active Case = confirmed - deaths - recovered
df['active'] = df['confirmed'] - df['deaths'] - df['recovered']

In [7]:
df['date'] = pd.to_datetime(df['date'])
country_list = list(df['country'].unique())

In [9]:
cols1=df.loc[:, ['country','date', 'confirmed']]
cols2 = cols1[cols1['country'] == 'Afghanistan' ]

Overall = ColumnDataSource(data=cols1)
Curr=ColumnDataSource(data=cols2)

#plot and the menu is linked with each other by this callback function
callback = CustomJS(args=dict(source=Overall, sc=Curr), code="""
var f = cb_obj.value
sc.data['date']=[]
sc.data['confirmed']=[]
for(var i = 0; i <= source.get_length(); i++){
	if (source.data['country'][i] == f){
		sc.data['date'].push(source.data['date'][i])
		sc.data['confirmed'].push(source.data['confirmed'][i])
	 }
}   
   
sc.change.emit();
""")

menu = Select(options=country_list,value='Afghanistan', title = 'Country')  # drop down menu
bokeh_p=figure(x_axis_label ='date', y_axis_label = 'confirmed', y_axis_type="linear",x_axis_type="datetime") #creating figure object 
bokeh_p.line(x='date', y='confirmed', color='green', source=Curr) # plotting the data using glyph circle
menu.js_on_change('value', callback) # calling the function on change of selection
layout=column(menu, bokeh_p) # creating the layout
show(layout)